In [4]:
from pathlib import Path 
import os
import torch
from transformers import BertTokenizerFast

In [11]:
print(f'@@Configuration START@@')

@@Configuration START@@


In [10]:
# Define configuration 
batch_size = 128
d_model = 512 
n_head = 8
max_len = 128
ffn_hidden = 1024 
n_layers=6 
drop_prob=0.1
epochs=300
init_lr = 1e-3
weight_decay = 5e-4
clip = 1

print(f'batch_size: {batch_size}')
print(f'd_model: {d_model}')
print(f'n_head: {n_head}') 
print(f'max_len: {max_len}') 
print(f'ffn_hidden: {ffn_hidden}')
print(f'n_layers: {n_layers}')
print(f'drop_prob: {drop_prob}')
print(f'epochs: {epochs}')
print(f'init_lr: {init_lr}')
print(f'weight_decay: {weight_decay}')
print(f'clip: {clip}')

batch_size: 128
d_model: 256
n_head: 2
max_len: 128
ffn_hidden: 64
n_layers: 6
drop_prob: 0.1
epochs: 300
init_lr: 0.001
weight_decay: 0.0005
clip: 1


In [ ]:
# Configure Device 


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA as device")
else:
    # Check that MPS is available
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                  "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                  "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
        print("Using CPU as device")
    else:
        device = torch.device("mps")
        print("Using MPS as device")
    
torch.set_default_device(device)

In [ ]:
# Define tokenizers
TOKENIZERS_PARALLELISM = True

kr_tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
en_tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")

enc_voc_size = kr_tokenizer.vocab_size
dec_voc_size = en_tokenizer.vocab_size

print(f'Using kr_tokenizer: kykim/bert-kor-base')
print(f'kr_tokenizer_voc_size(enc_voc_size): {enc_voc_size}')

print(f'Using en_tokenizer: google-bert/bert-base-uncased')
print(f'en_tokenizer_voc_size(dec_voc_size): {dec_voc_size}')

# Define some variables that are going to be used in future
src_pad_token = kr_tokenizer.pad_token_id
trg_pad_token = en_tokenizer.pad_token_id
trg_sos_token = en_tokenizer.sep_token_id

print(f'src_pad_token: {src_pad_token}')
print(f'trg_pad_token: {trg_pad_token}')
print(f'trg_sos_token: {trg_sos_token}')

In [ ]:
# Test tokenizers 
# tmp_kr_sentence = "오늘 하교길에 길고양이를 보았는데, 너무 귀여워서 집에 데려가고 싶었다. 하지만 그러지는 않았다."
# tmp_en_sentence = "The cat I saw during heading home today was so cute, that I wanted to bring it to home."

# tmp_kr_tokenized = kr_tokenizer(tmp_kr_sentence, add_special_tokens=True, padding="max_length", max_length=256, truncation=True)
# tmp_en_tokenized = en_tokenizer(tmp_en_sentence, add_special_tokens=True, padding="max_length", max_length=256, truncation=True)

# print(kr_tokenizer.convert_ids_to_tokens(tmp_kr_tokenized.input_ids))
# print(en_tokenizer.convert_ids_to_tokens(tmp_en_tokenized.input_ids))

# print(kr_tokenizer.decode(tmp_kr_tokenized.input_ids, skip_special_tokens=True))

# check if both tokenizer has pad token 
# print(kr_tokenizer.pad_token)
# print(en_tokenizer.pad_token)

In [21]:
# Define path configuration for the project 
project_dir = Path(os.getcwd()).parent
rawdata_dir = project_dir / "data"
data_dir = project_dir / "preprocessed"

rawdata_dir.mkdir(parents=True, exist_ok=True)
data_dir.mkdir(parents=True, exist_ok=True)

print(f'project_dir: {project_dir}')
print(f'rawdata_dir: {rawdata_dir}')
print(f'data_dir: {data_dir}')

project_dir: /Users/ball/Documents/workspace/transformer-tutorial
rawdata_dir: /Users/ball/Documents/workspace/transformer-tutorial/data
data_dir: /Users/ball/Documents/workspace/transformer-tutorial/preprocessed


In [ ]:
print(f'@@Configuration END@@')